# Lab N
### Morphology

Skimage morphology operators' documentation:
https://scikit-image.org/docs/dev/api/skimage.morphology.html

In [ ]:
from commonfunctions import *
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from skimage.measure import find_contours
from skimage.draw import rectangle

# Show the figures / plots inside the notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
'''
1) Erosion / Dilation:
    - Load "coins.jpg" from the img folder
    - Convert the image to binary
    - Write 2 functions each applying 1 of the erosion/dilation algorithms (with adjustable square window size) manually to the binary image
    - Use Skimage's "binary_erosion" & "binary_dilation" & show the results
'''
def binary(img):
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            img[i][j] = 1 if img[i][j] >= 0.5 else 0
    return img

def erosion(img, window_size, sE=None):
    # copy of the original image 
    original = img.copy()
    
    # output image
    eroded = np.zeros(img.shape)
    
    # setting the structural element by either the input or square filter of ones by window size
    se = sE if sE else np.ones((window_size, window_size))
    
    # length of side of filter
    seSize = len(se)
    
    # the index at which the original image starts in the padded image
    start = int(seSize / 2)
    
    # new size (with padding)
    newH = original.shape[1]+ 2 * seSize
    newW = original.shape[0]+ 2 * seSize
    
    # empty image of ones 
    img = np.ones((newW, newH))
    
    # setting the original image in the padded image so now we have the original image padded with ones
    img[start:start+original.shape[0], start:start+original.shape[1]] = original
    
    # looping on the original image and applying erosion
    for i in range(start, original.shape[0] + start):
        for j in range(start, original.shape[1] + start):
            I = i - start
            J = j - start
            mat = img[I: I + seSize, J: J + seSize]
            eroded[I][J] = 1 if (np.logical_and(mat, se) == se).all() else 0
            
    return eroded


def dilation(img, window_size, sE=None):
    # copy of the original image
    original = img.copy()
    
    # output image
    dilated = np.zeros(img.shape)
    
    # setting the structural element by either the input or square filter of ones by window size
    se = sE if sE else np.ones((window_size, window_size))
    seSize = len(se)
    
        
    start = int(seSize / 2)
    
    # new size (with padding)
    newH = original.shape[1]+ 2 * seSize
    newW = original.shape[0]+ 2 * seSize
    
    # empty image of zeros
    img = np.zeros((newW, newH))

    # setting the original image in the padded image so now we have the original image padded with zeros
    img[start:start+original.shape[0], start:start+original.shape[1]] = original
    
    # looping on the original image and applying dilation
    for i in range(start, original.shape[0] + start):
        for j in range(start, original.shape[1] + start):
            I = i - start
            J = j - start
            mat = img[I: I + seSize, J: J + seSize]
            dilated[I][J] = 1 if (np.logical_and(mat, se)).any() else 0
    return dilated        
    
# sE = [[1, 0, 1],[0,1,0],[1,0,1]]
size = 3

img = binary(rgb2gray(io.imread('img/coins.jpg')))
eroded = erosion(img, size)
dilated = dilation(img, size)
show_images([img, eroded, binary_erosion(img)], ["Original","My Erosion","Skimage's erosion"])
show_images([img, dilated, binary_dilation(img)], ["Original","My Dilation","Skimage's dilation"])

In [ ]:
'''
2) Case Study: Credit Card Number Extraction
    - Load "card.jpg" from the img folder
    - Convert the image to binary by brightness thresholding maintaining the card number details for extraction
    - Use your erosion/dilation functions, or Skimage's functions only if you didn't implement them, to apply closing
        Note: The goal of this step is to join the card number into 4 sets of connected pixels
    - Use Skimage's "find_contours" to get the bounding boxes for the credit card's numbers by filtering on aspect ratio (between 2.5 & 3.5)
        Note: When using "find_contours(img, 0.8)" the o/p is a list of contours each having:
                - contour[:,0] being the Y values of the contour perimeter points
                - contour[:,1] being the X values of the contour perimeter points
'''
def closing(img):
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            img[i][j] = 1 if img[i][j] > 0.65 else 0
    return erosion(dilation(img, 19), 19)


img = rgb2gray(io.imread('img/card.jpg'))

closed = closing(img)


array = find_contours(closed, 0.8)

#When provided with the correct format of the list of bounding_boxes, this section will set all pixels inside boxes in img_with_boxes
bounding_boxes = []
for i in array:
    y = i[:, 0]
    x = i[:, 1]
    bounding_boxes.append([int(min(x)), int(max(x)), int(min(y)), int(max(y))])
    
bounding_boxes = bounding_boxes[1:]
img_with_boxes = np.zeros(img.shape)#closed.copy()

for box in bounding_boxes:
    [Xmin, Xmax, Ymin, Ymax] = box
    
    rr, cc = rectangle(start = (Ymin,Xmin), end = (Ymax,Xmax), shape=img_with_boxes.shape)

    img_with_boxes[np.array(rr,dtype=np.uint32), np.array(cc,dtype=np.uint32)] = 1 #set color white
    
show_images([img, closed, img_with_boxes], ["Original", "closed", "image with boxes"])

In [ ]:
'''
3) Skeletonization:
    - Load "horse.jpg" from the img folder 
    - Use a binary threshold of 0.9 to remove invisible noise in the background
    - Use Skimage's "skeletonize(image)" to acquire the image's skeleton & show it
    - Use Skimage's "thin(image, max_iter)" with values of max_iter 5/10/15/20 & show it
'''
def skeletonization(img):
    origin = img.copy()
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            img[i][j] = 1 if img[i][j] >= 0.9 else 0
    show_images([origin, img, skeletonize(img)], ["Original", "removed noise", "skeleton"])
    show_images([thin(img, 5),thin(img, 10),thin(img, 15),thin(img, 20)], ["5","10","15","20"])

img = rgb2gray(io.imread("img/horse.jpg"))
skeletonization(img)